In [168]:
import numpy as np
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [169]:
# 사용할 노선과 합쳐야 하는 노선 목록(중복역은 합계 후 중복 삭제예정)
# 1호선:서울지하철1호선(청량리~서울역), 경원선(소요산~청량리), 경부선(서울~천안/서동탄/광명), 장항선(천안~신창), 경인선(구로~인천)
# 2호선, 5호선, 6호선, 7호선, 8호선
# 3호선: 서울지하철3호선(지축~오금), 일산선(대화~지축)
# 4호선: 서울지하철4호선(당고개~남태령), 과천선(남태령~금정), 안산선(금정~오이도)
# 9호선: 9호선, 9호선(2~3)단계
# 수인분당선: 수인선(인천~수원), 분당선()
# 공항철도
# 신림선, 우이신설선
# 신분당선 데이터 없음, 김포 골드라인 없음, 용인 에버 없음

# 호선 분리하기 ex) 1호선, 경원선, 수인선 ......

# 월별 평일 승하차 각각 합계, 월별 주말 승하차 각각 합계

# 월별로 처리한 데이터 csv로 각 년도 폴더에 저장

In [170]:
# 2020 csv 열기
df = pd.read_csv('data/2020.csv', encoding='euc-kr', sep=',')
df.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20200101,1호선,종각,20427,16301,20200104
1,20200101,1호선,시청,12126,10516,20200104
2,20200101,우이신설선,신설동,892,828,20200104
3,20200101,우이신설선,보문,917,855,20200104
4,20200101,우이신설선,성신여대입구(돈암),2010,2363,20200104


In [171]:
# 결측치 확인
df.isna().sum().sum() 

0

In [172]:
# 등록일자 삭제
df.drop(['등록일자'], axis=1, inplace=True)
df.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수
0,20200101,1호선,종각,20427,16301
1,20200101,1호선,시청,12126,10516
2,20200101,우이신설선,신설동,892,828
3,20200101,우이신설선,보문,917,855
4,20200101,우이신설선,성신여대입구(돈암),2010,2363


In [173]:
df['승하차총승객수'] = df['승차총승객수'] + df['하차총승객수']
df.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,승하차총승객수
0,20200101,1호선,종각,20427,16301,36728
1,20200101,1호선,시청,12126,10516,22642
2,20200101,우이신설선,신설동,892,828,1720
3,20200101,우이신설선,보문,917,855,1772
4,20200101,우이신설선,성신여대입구(돈암),2010,2363,4373


In [174]:
# 역명, 노선명 --> 지하철역, 호선명
df = df.rename(columns={'역명': '지하철역', '노선명':'호선명'})
df.head()

,사용일자,호선명,지하철역,승차총승객수,하차총승객수,승하차총승객수
0,20200101,1호선,종각,20427,16301,36728
1,20200101,1호선,시청,12126,10516,22642
2,20200101,우이신설선,신설동,892,828,1720
3,20200101,우이신설선,보문,917,855,1772
4,20200101,우이신설선,성신여대입구(돈암),2010,2363,4373


In [175]:
# ()제거
df['지하철역'] = df['지하철역'].str.replace('(', ' ',regex=False,).str.split().str[0]

In [176]:
# 노선 이름 보기
df['호선명'].unique()

array(['1호선', '우이신설선', '9호선2~3단계', '공항철도 1호선', '9호선', '8호선', '7호선', '6호선',
       '5호선', '경강선', '수인선', '경춘선', '경의선', '장항선', '중앙선', '일산선', '분당선',
       '과천선', '안산선', '경원선', '경인선', '경부선', '4호선', '3호선', '2호선'],
      dtype=object)

In [177]:
# 사용일자를 datetime 타입으로 변환
df['사용일자'] = pd.to_datetime(df['사용일자'], format='%Y%m%d')

# 주말과 평일을 분리
is_weekend = df['사용일자'].dt.weekday.isin([5, 6])  # 주말인지 여부를 판단하는 조건식
weekend_df = df[is_weekend]  # 주말 데이터프레임
weekday_df = df[~is_weekend]  # 평일 데이터프레임

In [178]:
weekend_df.tail()

,사용일자,호선명,지하철역,승차총승객수,하차총승객수,승하차총승객수
214659,2020-12-27,6호선,대흥,1953,2023,3976
214660,2020-12-27,6호선,광흥창,1983,1999,3982
214661,2020-12-27,6호선,상수,2312,2726,5038
214662,2020-12-27,6호선,합정,4258,3832,8090
214663,2020-12-27,6호선,망원,4691,5016,9707


In [179]:
weekday_df.tail()

,사용일자,호선명,지하철역,승차총승객수,하차총승객수,승하차총승객수
217050,2020-12-31,경의선,신촌,541,633,1174
217051,2020-12-31,경의선,서울역,2512,3329,5841
217052,2020-12-31,경원선,소요산,1522,1401,2923
217053,2020-12-31,경원선,동두천,1379,1589,2968
217054,2020-12-31,경원선,동두천중앙,2155,2219,4374


- 월별 평일의 승하차 합계

In [180]:
# 사용일자를 월로 변환
weekday_df['월'] = weekday_df['사용일자'].dt.month

# 월별로 데이터프레임 나누기
monthly_dfs = {}
for month, group in weekday_df.groupby('월'):
    monthly_dfs[month] = group

# 각 월별 데이터프레임 출력
for month, df in monthly_dfs.items():
    print(f"\n{month}월 데이터프레임:")
    print(df.head())


1월 데이터프레임:
        사용일자    호선명    지하철역  승차총승객수  하차총승객수  승하차총승객수  월
0 2020-01-01    1호선      종각   20427   16301    36728  1
1 2020-01-01    1호선      시청   12126   10516    22642  1
2 2020-01-01  우이신설선     신설동     892     828     1720  1
3 2020-01-01  우이신설선      보문     917     855     1772  1
4 2020-01-01  우이신설선  성신여대입구    2010    2363     4373  1

2월 데이터프레임:
            사용일자  호선명  지하철역  승차총승객수  하차총승객수  승하차총승객수  월
19493 2020-02-03  1호선   서울역   48307   46039    94346  2
19494 2020-02-03  1호선    시청   25019   25472    50491  2
19495 2020-02-03  1호선    종각   43754   42847    86601  2
19496 2020-02-03  1호선  종로3가   29601   28159    57760  2
19497 2020-02-03  1호선  종로5가   24251   24394    48645  2

3월 데이터프레임:
            사용일자  호선명  지하철역  승차총승객수  하차총승객수  승하차총승객수  월
36019 2020-03-02  7호선    남성    8849    8235    17084  3
36020 2020-03-02  1호선   서울역   30750   29308    60058  3
36021 2020-03-02  1호선    시청   17804   18180    35984  3
36022 2020-03-02  1호선    종각   28844   28043    56887  3
36023 2020-0

In [181]:
january_df = monthly_dfs.get(1)  # 1월의 데이터프레임 가져오기
january_df.head()

,사용일자,호선명,지하철역,승차총승객수,하차총승객수,승하차총승객수,월
0,2020-01-01,1호선,종각,20427,16301,36728,1
1,2020-01-01,1호선,시청,12126,10516,22642,1
2,2020-01-01,우이신설선,신설동,892,828,1720,1
3,2020-01-01,우이신설선,보문,917,855,1772,1
4,2020-01-01,우이신설선,성신여대입구,2010,2363,4373,1


In [182]:
# 월별로 그룹화하여 합계 계산
result_dfs = {}  # 결과를 담을 딕셔너리

for month, df in monthly_dfs.items():
    grouped_df = df.groupby(['사용일자','호선명', '지하철역'])['승차총승객수', '하차총승객수', '승하차총승객수'].sum().reset_index()
    result_dfs[month] = grouped_df

# 결과 출력
for month, result_df in result_dfs.items():
    print(f"{month}월 결과:")
    print(result_df.head())
    print()

1월 결과:
        사용일자  호선명 지하철역  승차총승객수  하차총승객수  승하차총승객수
0 2020-01-01  1호선  동대문    8822    9558    18380
1 2020-01-01  1호선  동묘앞    7580    7998    15578
2 2020-01-01  1호선  서울역   36470   29895    66365
3 2020-01-01  1호선   시청   12126   10516    22642
4 2020-01-01  1호선  신설동    6820    6589    13409

2월 결과:
        사용일자  호선명 지하철역  승차총승객수  하차총승객수  승하차총승객수
0 2020-02-03  1호선  동대문   11981   12001    23982
1 2020-02-03  1호선  동묘앞    7572    7863    15435
2 2020-02-03  1호선  서울역   48307   46039    94346
3 2020-02-03  1호선   시청   25019   25472    50491
4 2020-02-03  1호선  신설동   15527   15344    30871

3월 결과:
        사용일자  호선명 지하철역  승차총승객수  하차총승객수  승하차총승객수
0 2020-03-02  1호선  동대문    8780    8767    17547
1 2020-03-02  1호선  동묘앞    5846    6042    11888
2 2020-03-02  1호선  서울역   30750   29308    60058
3 2020-03-02  1호선   시청   17804   18180    35984
4 2020-03-02  1호선  신설동   11925   11750    23675

4월 결과:
        사용일자  호선명 지하철역  승차총승객수  하차총승객수  승하차총승객수
0 2020-04-01  1호선  동대문    9482    9373    18855
1 2020-04

In [183]:
import os

# 저장할 폴더 경로
folder_path = 'data/월별 평일'
os.makedirs(folder_path, exist_ok=True)  # 폴더가 없는 경우 생성

# 월별로 데이터프레임 저장하기
for month, df in result_dfs.items():
    filename = f"{month}월_평일.csv"
    file_path = os.path.join(folder_path, filename)  # 폴더 경로와 파일명을 결합
    df.to_csv(file_path, index=False)
    print(f"{file_path} 파일이 저장되었습니다.")

data/월별 평일\1월_평일.csv 파일이 저장되었습니다.
data/월별 평일\2월_평일.csv 파일이 저장되었습니다.
data/월별 평일\3월_평일.csv 파일이 저장되었습니다.
data/월별 평일\4월_평일.csv 파일이 저장되었습니다.
data/월별 평일\5월_평일.csv 파일이 저장되었습니다.
data/월별 평일\6월_평일.csv 파일이 저장되었습니다.
data/월별 평일\7월_평일.csv 파일이 저장되었습니다.
data/월별 평일\8월_평일.csv 파일이 저장되었습니다.
data/월별 평일\9월_평일.csv 파일이 저장되었습니다.
data/월별 평일\10월_평일.csv 파일이 저장되었습니다.
data/월별 평일\11월_평일.csv 파일이 저장되었습니다.
data/월별 평일\12월_평일.csv 파일이 저장되었습니다.


In [184]:
df_99 = pd.read_csv('data/월별 평일/1월_평일.csv')
df_99.head()

,사용일자,호선명,지하철역,승차총승객수,하차총승객수,승하차총승객수
0,2020-01-01,1호선,동대문,8822,9558,18380
1,2020-01-01,1호선,동묘앞,7580,7998,15578
2,2020-01-01,1호선,서울역,36470,29895,66365
3,2020-01-01,1호선,시청,12126,10516,22642
4,2020-01-01,1호선,신설동,6820,6589,13409


- 월별 주말 승하차 합계

In [185]:
# 사용일자를 월로 변환
weekend_df['월'] = weekend_df['사용일자'].dt.month

# 월별로 데이터프레임 나누기
monthly_dfs = {}
for month, group in weekend_df.groupby('월'):
    monthly_dfs[month] = group

# 각 월별 데이터프레임 출력
for month, df in monthly_dfs.items():
    print(f"\n{month}월 데이터프레임:")
    print(df.head())


1월 데이터프레임:
           사용일자    호선명    지하철역  승차총승객수  하차총승객수  승하차총승객수  월
1774 2020-01-04  우이신설선  북한산보국문    5117    4407     9524  1
1775 2020-01-04    1호선     서울역   48376   46823    95199  1
1776 2020-01-04    1호선      시청   22370   23449    45819  1
1777 2020-01-04    1호선      종각   35345   33183    68528  1
1778 2020-01-04    1호선    종로3가   34613   33921    68534  1

2월 데이터프레임:
            사용일자  호선명  지하철역  승차총승객수  하차총승객수  승하차총승객수  월
18312 2020-02-01  1호선    시청   13966   14315    28281  2
18313 2020-02-01  1호선    종각   24014   23628    47642  2
18314 2020-02-01  1호선  종로3가   25613   24969    50582  2
18315 2020-02-01  1호선  종로5가   19922   19765    39687  2
18316 2020-02-01  1호선   동대문   11346   11210    22556  2

3월 데이터프레임:
            사용일자       호선명    지하철역  승차총승객수  하차총승객수  승하차총승객수  월
35430 2020-03-01  공항철도 1호선      운서    3621    3795     7416  3
35431 2020-03-01     우이신설선     신설동     636     646     1282  3
35432 2020-03-01     우이신설선      보문     564     542     1106  3
35433 2020-03-01     우

In [186]:
january_df = monthly_dfs.get(1)  # 1월의 데이터프레임 가져오기
january_df.head()

,사용일자,호선명,지하철역,승차총승객수,하차총승객수,승하차총승객수,월
1774,2020-01-04,우이신설선,북한산보국문,5117,4407,9524,1
1775,2020-01-04,1호선,서울역,48376,46823,95199,1
1776,2020-01-04,1호선,시청,22370,23449,45819,1
1777,2020-01-04,1호선,종각,35345,33183,68528,1
1778,2020-01-04,1호선,종로3가,34613,33921,68534,1


In [187]:
# 월별로 그룹화하여 합계 계산
result_dfs2 = {}  # 결과를 담을 딕셔너리

for month, df in monthly_dfs.items():
    grouped_df = df.groupby(['사용일자','호선명', '지하철역'])['승차총승객수', '하차총승객수', '승하차총승객수'].sum().reset_index()
    result_dfs2[month] = grouped_df

# 결과 출력
for month, result_df in result_dfs2.items():
    print(f"{month}월 결과:")
    print(result_df.head())
    print()

1월 결과:
        사용일자  호선명 지하철역  승차총승객수  하차총승객수  승하차총승객수
0 2020-01-04  1호선  동대문   13392   15219    28611
1 2020-01-04  1호선  동묘앞   13077   13909    26986
2 2020-01-04  1호선  서울역   48376   46823    95199
3 2020-01-04  1호선   시청   22370   23449    45819
4 2020-01-04  1호선  신설동   12452   11889    24341

2월 결과:
        사용일자  호선명 지하철역  승차총승객수  하차총승객수  승하차총승객수
0 2020-02-01  1호선  동대문   11346   11210    22556
1 2020-02-01  1호선  동묘앞   10003   10716    20719
2 2020-02-01  1호선  서울역   34375   33070    67445
3 2020-02-01  1호선   시청   13966   14315    28281
4 2020-02-01  1호선  신설동   10517   10085    20602

3월 결과:
        사용일자  호선명 지하철역  승차총승객수  하차총승객수  승하차총승객수
0 2020-03-01  1호선  동대문    4304    5054     9358
1 2020-03-01  1호선  동묘앞    5954    6020    11974
2 2020-03-01  1호선  서울역   10457   10389    20846
3 2020-03-01  1호선   시청    3980    3937     7917
4 2020-03-01  1호선  신설동    4440    4297     8737

4월 결과:
        사용일자  호선명 지하철역  승차총승객수  하차총승객수  승하차총승객수
0 2020-04-04  1호선  동대문    8208    7575    15783
1 2020-04

In [188]:
import os

# 저장할 폴더 경로
folder_path2 = 'data/월별 주말'
os.makedirs(folder_path2, exist_ok=True)  # 폴더가 없는 경우 생성

# 월별로 데이터프레임 저장하기
for month, df in result_dfs.items():
    filename = f"{month}월_주말.csv"
    file_path = os.path.join(folder_path2, filename)  # 폴더 경로와 파일명을 결합
    df.to_csv(file_path, index=False)
    print(f"{file_path} 파일이 저장되었습니다.")

data/월별 주말\1월_주말.csv 파일이 저장되었습니다.
data/월별 주말\2월_주말.csv 파일이 저장되었습니다.
data/월별 주말\3월_주말.csv 파일이 저장되었습니다.
data/월별 주말\4월_주말.csv 파일이 저장되었습니다.
data/월별 주말\5월_주말.csv 파일이 저장되었습니다.
data/월별 주말\6월_주말.csv 파일이 저장되었습니다.
data/월별 주말\7월_주말.csv 파일이 저장되었습니다.
data/월별 주말\8월_주말.csv 파일이 저장되었습니다.
data/월별 주말\9월_주말.csv 파일이 저장되었습니다.
data/월별 주말\10월_주말.csv 파일이 저장되었습니다.
data/월별 주말\11월_주말.csv 파일이 저장되었습니다.
data/월별 주말\12월_주말.csv 파일이 저장되었습니다.


In [189]:
df_999 = pd.read_csv('data/월별 주말/1월_주말.csv')
df_999.head()

,사용일자,호선명,지하철역,승차총승객수,하차총승객수,승하차총승객수
0,2020-01-01,1호선,동대문,8822,9558,18380
1,2020-01-01,1호선,동묘앞,7580,7998,15578
2,2020-01-01,1호선,서울역,36470,29895,66365
3,2020-01-01,1호선,시청,12126,10516,22642
4,2020-01-01,1호선,신설동,6820,6589,13409


In [190]:
df['사용일자'].dtype

dtype('<M8[ns]')